In [2]:
import socket
import sys
import tkinter as tk
from tkinter import *
from tkinter import filedialog
from PIL import ImageTk, Image
from tkinter import messagebox
from tkinter.filedialog import askopenfile
import cv2
import numpy as np
import struct
window = tk.Tk()
window.geometry("700x700")
window.title("Login")
window.resizable(False,False)
# load the image file
try:
    bg_image = tk.PhotoImage(file='bgg.png')
except tk.TclError as e:
    print(f"Error loading image file: {e}")
else:
    # create a label with the PhotoImage as the background image
    bg_label = tk.Label(window, image=bg_image)
    bg_label.place(x=0, y=0, relwidth=1, relheight=1)

username_value = tk.StringVar()
password_value = tk.StringVar()

def capture():
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    if ret:
        cv2.imwrite("capture.png", frame)
        filepath = "capture.png"
        extension = filepath.split('.', 1)
        print("File Type : "+extension[1])
        clientSock.sendto(extension[1].encode(), ("127.0.0.1", 20001))
        img=cv2.imread (filepath)
        cv2.imshow('img',img)
        img_encode=cv2.imencode ("."+extension[1], img)[1]
        data_encode=np.array (img_encode)
        data=data_encode.tostring()
        print("Length of Data : "+ str(len(data)))
        print("Encoded Byte Data : ")
        print(data)
        fhead=struct.pack ("l", len (data))
        clientSock.sendto (fhead, ("127.0.0.1", 20001))
        for i in range (len (data) //1024 + 1):
            if 1024 * (i + 1)>len (data):
                clientSock.sendto (data [1024 * i:], ("127.0.0.1", 20001))
            else:
                clientSock.sendto (data [1024 * i:1024 * (i + 1)], ("127.0.0.1", 20001))
        cv2.waitKey(0)
        cv2.destroyAllWindows()

def upload():
    filepath=filedialog.askopenfilename(filetypes=[("image", ".jpeg"),("image", ".png"),("image", ".jpg")])
    print("File location : "+filepath)
    extension = filepath.split('.', 1)
    print("File Type : "+extension[1])
    clientSock.sendto(extension[1].encode(), ("127.0.0.1", 20001))
    img=cv2.imread (filepath)
    cv2.imshow('img',img)
    img_encode=cv2.imencode ("."+extension[1], img)[1]
    data_encode=np.array (img_encode)
    data=data_encode.tostring()
    #data=cv2.imencode("."+extension[1],img)[1].tostring()
    print("Length of Data : "+ str(len(data)))
    print("Encoded Byte Data : ")
    print(data)
    #Defining file headers,Packaging into a structure
    
    fhead=struct.pack ("l", len (data))
    
    #Send file header:
    
    clientSock.sendto (fhead, ("127.0.0.1", 20001))
    
    #Cyclic picture stream
    
    for i in range (len (data) //1024 + 1):
        if 1024 * (i + 1)>len (data):
            #checks if any data is left and sends it irrespective of size
            clientSock.sendto (data [1024 * i:], ("127.0.0.1", 20001))
        else:
            #sends data with a size of 1024
            clientSock.sendto (data [1024 * i:1024 * (i + 1)], ("127.0.0.1", 20001))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def clicked():
    messagebox.showwarning('Invalid', 'Incorrect Password!')

def submit_login():
    username=username_value.get()
    password=password_value.get()
    if(username=="client" and password=="client"):
        print("You are now logged in")
        window.destroy()
        global clientSock
        clientSock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        clientSock.sendto("Client connection established".encode(), ("127.0.0.1", 20001))
        print("Connection established, sent message to server")
        upload_file=tk.Tk()
        upload_file.geometry("700x700")
        upload_file.title("Upload an Image")
        upload_file.resizable(False,False)
        try:
            bimg = tk.PhotoImage(file='bg2.png')
        except tk.TclError as e:
            messagebox.showwarning('Invalid', 'Check Again!')
        else:
            # create a label with the PhotoImage as the background image
            bgl = tk.Label(upload_file, image=bimg)
            bgl.place(x=0, y=0, relwidth=1, relheight=1)
        tk.Label(upload_file, text="Upload an Image", font="Courier 20 bold").place(relx=0.5, rely=0.17, anchor='center')
        peach = '#F47C7C'

        tk.Button(text="Upload an Image", font="Courier 15", width=20, command=upload, bg=peach, fg='white').place(relx=0.5, rely=0.65, anchor='center')

        tk.Button(text="Capture an Image", font="Courier 15", width=20, command=capture, bg=peach, fg='white').place(relx=0.5, rely=0.75, anchor='center')

        upload_file.mainloop()
        
    else:
        clicked()
tk.Label(window, text="Client Login", font="Courier 20 bold").place(relx=0.5, rely=0.1, anchor='center')

username = tk.Label(window, text="Username", font="Courier 15")
username.place(relx=0.25, rely=0.25)
password = tk.Label(window, text="Password", font="Courier 15")
password.place(relx=0.25, rely=0.35)

username_box = tk.Entry(window, textvariable=username_value)
username_box.place(relx=0.55, rely=0.25, width=150, height=25)
password_box = tk.Entry(window, textvariable=password_value, show="*")
password_box.place(relx=0.55, rely=0.35, width=150, height=25)
peach = '#F47C7C'
tk.Button(text="Login", command=submit_login, font="Courier 15", bg=peach, fg='white').place(relx=0.5, rely=0.5, anchor=tk.CENTER)

window.grid_columnconfigure(10, weight=1)
window.grid_columnconfigure(15, weight=1)

window.mainloop()